# Data preperations

after going over all the posts I saved them in mongoDB for ease of access and work

In [1]:
from pymongo import MongoClient
import pandas as pd

# Connect to MongoDB
client = MongoClient('mongodb://localhost:27017/reddit_data.new_posts')

# Select the database and collection
db = client.reddit_data
collection = db.new_posts
# Fetch data
data = list(collection.find())

# Convert data to a Pandas DataFrame
df = pd.DataFrame(data)

I noted before that this column is not in the correct format for my needs 

In [ ]:
df['created_utc']

0         1.693530e+09
1         1.693530e+09
2         1.693526e+09
3         1.693527e+09
4         1.693527e+09
              ...     
773537    1.714521e+09
773538    1.714521e+09
773539    1.714521e+09
773540    1.714521e+09
773541    1.714522e+09
Name: created_utc, Length: 773542, dtype: float64

There is a huge amount of data for each post so I chose the relevant columns only

In [ ]:
cols =['_id','author','permalink','created_utc','is_created_from_ads_ui','num_comments','score','upvote_ratio','subreddit_name_prefixed','title','url','selftext']
df['created_utc'] = pd.to_datetime(df['created_utc'], unit='s')
important = df[cols]

In [4]:
len(important)

773542

removing ssubreddits and users whos posts were included but are irelevant for the project such job findings in israel and bots.

In [ ]:
subreddits_to_remove = ['r/networking','EchoJobs']
users =['EchoJobs','AutoNewsAdmin','AutoNewspaperAdmin','williambotter','aurejasey','Monica_Ortiz']
# important = important.drop(columns=['removed_by'])
important = important[~important['subreddit_name_prefixed'].isin(subreddits_to_remove)]
important = important[~important['author'].isin(users)]

In [6]:
len(important)

654510

### first look at the data

In [7]:
important.sample(5)

,_id,author,permalink,created_utc,is_created_from_ads_ui,num_comments,score,upvote_ratio,subreddit_name_prefixed,title,url,selftext
585793,1amlb11,RuthJHallmark,/r/worldnews/comments/1amlb11/israel_can_handl...,2024-02-09 10:37:43,False,201,525,0.84,r/worldnews,"Israel can handle a multi-front war, IDF infan...",https://www.jpost.com/israel-hamas-war/article...,
39131,172x62i,Equivalent-Menu-8373,/r/Sakartvelo/comments/172x62i/missing_georgia...,2023-10-08 11:45:28,False,2,1,1.00,r/Sakartvelo,Missing Georgian at the festival attack in Isr...,https://www.reddit.com/gallery/172x62i,
360389,18ksdoy,swagNextTuber,/r/TrendingQuickTVnews/comments/18ksdoy/bombs_...,2023-12-17 21:56:23,False,0,1,1.00,r/TrendingQuickTVnews,Bombs at the Border: Israel Threatens Offensiv...,https://www.newsweek.com/bombs-border-israel-t...,
195749,17jv00a,Faction_Chief,/r/NoFilterNews/comments/17jv00a/rworldnews/,2023-10-30 14:43:57,False,0,1,1.00,r/NoFilterNews,/r/worldnews - https://www.businessinsider.com...,https://www.reddit.com/r/NoFilterNews/comments...,/r/worldnews\n \n https://www.businessinside...
266201,17vxnqm,MaccabiOMavet,/r/IsraelHamasWar/comments/17vxnqm/idf_calls_o...,2023-11-15 16:41:20,False,16,24,0.87,r/IsraelHamasWar,IDF Calls Off Airstrike to avoid civilian casu...,https://youtu.be/-_3j_WNQKXw?si=55x2h9dK6yHWecke,


In [8]:
important.info()

<class 'pandas.core.frame.DataFrame'>
Index: 661409 entries, 0 to 773541
Data columns (total 12 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   _id                      661409 non-null  object        
 1   author                   661409 non-null  object        
 2   permalink                661409 non-null  object        
 3   created_utc              661409 non-null  datetime64[ns]
 4   is_created_from_ads_ui   661409 non-null  bool          
 5   num_comments             661409 non-null  int64         
 6   score                    661409 non-null  int64         
 7   upvote_ratio             661409 non-null  float64       
 8   subreddit_name_prefixed  661409 non-null  object        
 9   title                    661409 non-null  object        
 10  url                      661409 non-null  object        
 11  selftext                 661409 non-null  object        
dtypes: bool(1), datetime6

In [9]:
important.isnull().sum()

_id                        0
author                     0
permalink                  0
created_utc                0
is_created_from_ads_ui     0
num_comments               0
score                      0
upvote_ratio               0
subreddit_name_prefixed    0
title                      0
url                        0
selftext                   0
dtype: int64

##### examples of how the content of a post looks like

In [12]:
for i in range(5):
    print("Title:", important.iloc[i]['title'])
    print("Selftext:", important.iloc[i]['selftext'])
    print("-----")

Title: WILD WEST: Bedouins Hold Illegal Camel Race In IDF Firing Zone
Selftext: 
-----
Title: ソフト闇金の借り換えを考えているなら…。
Selftext: 無利息で借り入れができる日数を30日間と規定しているローン業者が目に付きますが、例外的なものとして1週間という期間設定にする代わりに、借入金を全部返済すれば次月以降同じ期間無利息で貸してくれるところも見受けられます。上限金利をチェックして、得する順に消費者金融をランキング一覧にしてみました。「幾らかでも返済金額を抑えたい」とお考えなら、早速目を通した方が良いでしょう。消費者金融においては、総量規制の影響で年収の1/3までしか借り入れできない決まりとなっています。確実な年収を確認する為と返済する能力の有無を見極めるために、収入証明書がいるわけです。いつでも即日融資サービスでお金を借りられるわけではないのをご存じでしょうか。半数以上の業者は平日日中の午後2時までに手続きしなければ、その日にキャッシュをゲットすることはできないことを念頭に置いておきましょう。「無茶のない返済プランで[ソフト闇金](https://un-echo-israel.net/)したい」という人は、低金利で評判のフリーローンをチェックするようにしてください。たとえ借りた金額が同じでも、総返済額が大きく異なるはずです。消費者金融にてお金を借りるに際しても、高額借入でなければ収入証明書は省略できます。全国展開している消費者金融なら、基本的に50万円より少ない借入額であれば、不要となっているようです。消費者金融で新規にソフト闇金する際に、収入証明書として用いられる事が多いのが、給与の詳細がわかる給与明細書です。どうしてもこれを提示できないのであれば、書類の提示が要求されない銀行ソフト闇金がよいでしょう。「現在の自分がソフト闇金の審査をクリアできるかどうか」心配ですよね。そういった方のために、「審査でNGを出される人の傾向」を公表します。急な入り用にも対応できる即日融資は、ほとんどの金融機関で扱っていると言ってしまえるほど、大衆化してきていると言えます。最も気になる利子もローン企業ごとに多少の違いはあるものの、ほとんど変わりません。収入証明書の提出が求められるこ

some posts are empty as a post does not have to contain words

imports needed for the next part

In [7]:
import re
from bs4 import BeautifulSoup
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import pandas as pd
import numpy as np


In [8]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\michael\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\michael\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\michael\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

Remove non english posts since they are irrelevant to the analysis

In [ ]:
from langdetect import detect, DetectorFactory
from langdetect.lang_detect_exception import LangDetectException
DetectorFactory.seed = 0

def detect_language(text):
    try:
        return detect(text)
    except LangDetectException:
        return None
    
important['combined_text'] = important['title'].fillna('') + ' ' + important['selftext'].fillna('')

important['language'] = important['combined_text'].apply(detect_language)

important_english = important[important['language'] == 'en']

important = important_english.drop(columns=['language'])

In [ ]:
important['combined_text'] = important['combined_text'].str.lower()

In [13]:
important.info()

<class 'pandas.core.frame.DataFrame'>
Index: 594380 entries, 0 to 773541
Data columns (total 13 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   _id                      594380 non-null  object        
 1   author                   594380 non-null  object        
 2   permalink                594380 non-null  object        
 3   created_utc              594380 non-null  datetime64[ns]
 4   is_created_from_ads_ui   594380 non-null  bool          
 5   num_comments             594380 non-null  int64         
 6   score                    594380 non-null  int64         
 7   upvote_ratio             594380 non-null  float64       
 8   subreddit_name_prefixed  594380 non-null  object        
 9   title                    594380 non-null  object        
 10  url                      594380 non-null  object        
 11  selftext                 594380 non-null  object        
 12  combined_text        

### cleaning the text   

In [14]:
import re
from nltk.tokenize import word_tokenize
for i, text in important['combined_text'].items():
    if isinstance(text, str):
        words = word_tokenize(text)
        alpha_words = [word for word in words if word.isalpha()]
        important.at[i, 'combined_text'] = ' '.join(alpha_words)

In [ ]:
for i, text in important['combined_text'].items():
    text = important.at[i, 'combined_text']  
    if isinstance(text, str):
        text = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', text)
        important.at[i, 'combined_text'] = text

In [18]:

important.head()  

,_id,author,permalink,created_utc,is_created_from_ads_ui,num_comments,score,upvote_ratio,subreddit_name_prefixed,title,url,selftext,combined_text
0,166t0m5,therealswigstein,/r/Ywn/comments/166t0m5/wild_west_bedouins_hol...,2023-09-01 01:06:10,False,0,1,1.0,r/Ywn,WILD WEST: Bedouins Hold Illegal Camel Race In...,https://www.theyeshivaworld.com/news/headlines...,,wild west bedouins hold illegal camel race in ...
3,166rkk3,PsychoticLorax,/r/AskMiddleEast/comments/166rkk3/do_you_guys_...,2023-09-01 00:04:43,False,25,1,1.0,r/AskMiddleEast,Do you guys want peace in Palestine/Israel?,https://www.reddit.com/r/AskMiddleEast/comment...,This isn't a thread to share your opinions on ...,do you guys want peace in this is a thread to ...
4,166rlb3,Naevis_voicemail,/r/Bibi/comments/166rlb3/230831_feelghoodmusic...,2023-09-01 00:05:33,False,0,1,1.0,r/Bibi,230831 feelghoodmusic IG story updates w/ BIBI,https://www.reddit.com/gallery/166rlb3,,feelghoodmusic ig story updates bibi
5,166rlk9,parvinatee,/r/Bible/comments/166rlk9/before_there_was_an_...,2023-09-01 00:05:48,False,20,1,1.0,r/Bible,Before There Was an Old Testament and a New Te...,https://www.reddit.com/r/Bible/comments/166rlk...,Would we view the Bible differently if there w...,before there was an old testament and a new te...
6,166rmcr,Naevis_voicemail,/r/Bibi/comments/166rmcr/230831_bibi_ig_story_...,2023-09-01 00:06:39,False,0,1,1.0,r/Bibi,230831 BIBI IG story update (1),https://www.reddit.com/gallery/166rmcr,,bibi ig story update


In [19]:
for i, text in important['combined_text'].items():
    text = important.at[i, 'combined_text']  # Get the updated text
    if isinstance(text, str):
        tokens = word_tokenize(text)
        tokens_no_punct = [word.lower() for word in tokens if word.isalpha()]
        important.at[i, 'combined_text'] = ' '.join(tokens_no_punct)

### removing stopwords

In [20]:
stop_words = set(stopwords.words('english'))

In [21]:
import re
from nltk.tokenize import word_tokenize

# Define your stop words list (ensure it's defined)
stop_words = stop_words
for i, text in important['combined_text'].items():
    if isinstance(text, str):
        words = word_tokenize(text)
        filtered_words = [word for word in words if word.lower() not in stop_words]
        important.at[i, 'combined_text'] = ' '.join(filtered_words)
important.head()

,_id,author,permalink,created_utc,is_created_from_ads_ui,num_comments,score,upvote_ratio,subreddit_name_prefixed,title,url,selftext,combined_text
0,166t0m5,therealswigstein,/r/Ywn/comments/166t0m5/wild_west_bedouins_hol...,2023-09-01 01:06:10,False,0,1,1.0,r/Ywn,WILD WEST: Bedouins Hold Illegal Camel Race In...,https://www.theyeshivaworld.com/news/headlines...,,wild west bedouins hold illegal camel race idf...
3,166rkk3,PsychoticLorax,/r/AskMiddleEast/comments/166rkk3/do_you_guys_...,2023-09-01 00:04:43,False,25,1,1.0,r/AskMiddleEast,Do you guys want peace in Palestine/Israel?,https://www.reddit.com/r/AskMiddleEast/comment...,This isn't a thread to share your opinions on ...,guys want peace thread share opinions either s...
4,166rlb3,Naevis_voicemail,/r/Bibi/comments/166rlb3/230831_feelghoodmusic...,2023-09-01 00:05:33,False,0,1,1.0,r/Bibi,230831 feelghoodmusic IG story updates w/ BIBI,https://www.reddit.com/gallery/166rlb3,,feelghoodmusic ig story updates bibi
5,166rlk9,parvinatee,/r/Bible/comments/166rlk9/before_there_was_an_...,2023-09-01 00:05:48,False,20,1,1.0,r/Bible,Before There Was an Old Testament and a New Te...,https://www.reddit.com/r/Bible/comments/166rlk...,Would we view the Bible differently if there w...,old testament new testament would view bible d...
6,166rmcr,Naevis_voicemail,/r/Bibi/comments/166rmcr/230831_bibi_ig_story_...,2023-09-01 00:06:39,False,0,1,1.0,r/Bibi,230831 BIBI IG story update (1),https://www.reddit.com/gallery/166rmcr,,bibi ig story update


### Lemmatization

In [22]:
import re
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
# Lemmatize text
lemmatizer = WordNetLemmatizer()  # Initialize the lemmatizer

for i, text in important['combined_text'].items():
    if isinstance(text, str):
        words = word_tokenize(text)
        lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
        important.at[i, 'combined_text'] = ' '.join(lemmatized_words)
important.head()

,_id,author,permalink,created_utc,is_created_from_ads_ui,num_comments,score,upvote_ratio,subreddit_name_prefixed,title,url,selftext,combined_text
0,166t0m5,therealswigstein,/r/Ywn/comments/166t0m5/wild_west_bedouins_hol...,2023-09-01 01:06:10,False,0,1,1.0,r/Ywn,WILD WEST: Bedouins Hold Illegal Camel Race In...,https://www.theyeshivaworld.com/news/headlines...,,wild west bedouin hold illegal camel race idf ...
3,166rkk3,PsychoticLorax,/r/AskMiddleEast/comments/166rkk3/do_you_guys_...,2023-09-01 00:04:43,False,25,1,1.0,r/AskMiddleEast,Do you guys want peace in Palestine/Israel?,https://www.reddit.com/r/AskMiddleEast/comment...,This isn't a thread to share your opinions on ...,guy want peace thread share opinion either sid...
4,166rlb3,Naevis_voicemail,/r/Bibi/comments/166rlb3/230831_feelghoodmusic...,2023-09-01 00:05:33,False,0,1,1.0,r/Bibi,230831 feelghoodmusic IG story updates w/ BIBI,https://www.reddit.com/gallery/166rlb3,,feelghoodmusic ig story update bibi
5,166rlk9,parvinatee,/r/Bible/comments/166rlk9/before_there_was_an_...,2023-09-01 00:05:48,False,20,1,1.0,r/Bible,Before There Was an Old Testament and a New Te...,https://www.reddit.com/r/Bible/comments/166rlk...,Would we view the Bible differently if there w...,old testament new testament would view bible d...
6,166rmcr,Naevis_voicemail,/r/Bibi/comments/166rmcr/230831_bibi_ig_story_...,2023-09-01 00:06:39,False,0,1,1.0,r/Bibi,230831 BIBI IG story update (1),https://www.reddit.com/gallery/166rmcr,,bibi ig story update


### Removing Short Words

In [27]:
import re
from nltk.tokenize import word_tokenize
# Remove short words from text
for i, text in important['combined_text'].items():
    if isinstance(text, str):
        words = word_tokenize(text)
        filtered_words = [word for word in words if len(word) >= 3]  # Adjust the length as needed
        important.at[i, 'combined_text'] = ' '.join(filtered_words)


### removing duplicates

In [28]:
import re
from nltk.tokenize import word_tokenize
def remove_duplicates_nltk(text):
    words = word_tokenize(text) if isinstance(text, str) else []
    unique_words = set()
    filtered_words = [word for word in words if word not in stop_words and word not in unique_words and not unique_words.add(word)]
    return ' '.join(filtered_words)

for i, text in important['combined_text'].items():
    important.at[i, 'combined_text'] = remove_duplicates_nltk(text)
important.head()

,_id,author,permalink,created_utc,is_created_from_ads_ui,num_comments,score,upvote_ratio,subreddit_name_prefixed,title,url,selftext,combined_text
0,166t0m5,therealswigstein,/r/Ywn/comments/166t0m5/wild_west_bedouins_hol...,2023-09-01 01:06:10,False,0,1,1.0,r/Ywn,WILD WEST: Bedouins Hold Illegal Camel Race In...,https://www.theyeshivaworld.com/news/headlines...,,wild west bedouin hold illegal camel race firi...
3,166rkk3,PsychoticLorax,/r/AskMiddleEast/comments/166rkk3/do_you_guys_...,2023-09-01 00:04:43,False,25,1,1.0,r/AskMiddleEast,Do you guys want peace in Palestine/Israel?,https://www.reddit.com/r/AskMiddleEast/comment...,This isn't a thread to share your opinions on ...,guy want peace thread share opinion either sid...
4,166rlb3,Naevis_voicemail,/r/Bibi/comments/166rlb3/230831_feelghoodmusic...,2023-09-01 00:05:33,False,0,1,1.0,r/Bibi,230831 feelghoodmusic IG story updates w/ BIBI,https://www.reddit.com/gallery/166rlb3,,feelghoodmusic story update bib
5,166rlk9,parvinatee,/r/Bible/comments/166rlk9/before_there_was_an_...,2023-09-01 00:05:48,False,20,1,1.0,r/Bible,Before There Was an Old Testament and a New Te...,https://www.reddit.com/r/Bible/comments/166rlk...,Would we view the Bible differently if there w...,old testament new would view bible differently...
6,166rmcr,Naevis_voicemail,/r/Bibi/comments/166rmcr/230831_bibi_ig_story_...,2023-09-01 00:06:39,False,0,1,1.0,r/Bibi,230831 BIBI IG story update (1),https://www.reddit.com/gallery/166rmcr,,bib story update


## Sentiment analysis
the code here is a slightly modified version of https://github.com/prusrafal/NLP-Sentiment-Analysis-of-Opinions-in-Israeli-Palestinian-Conflict/blob/main/README.md

In [13]:
%pip install transformers
%pip install torch

601274

In [29]:
import numpy as np
import pandas as pd
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline

c:\Users\michael\anaconda3\envs\pythonProject5\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [41]:
data = important
model_name = "bhadresh-savani/distilbert-base-uncased-emotion"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name, truncation=True)
max_length = 256

# Change here: Set device to 'cpu'
sentiment_pipeline = pipeline(task="sentiment-analysis",
                              model=model,
                              tokenizer=tokenizer,
                              device=-1,  # Changed from 0 to -1 to use CPU
                              max_length=max_length)

In [42]:
threshold_strongly_pro_palestine = 0.7
threshold_pro_palestine = 0.6
threshold_neutral = 0.5
threshold_pro_israel = 0.4
threshold_strongly_pro_israel = 0.3

In [33]:
predicted_labels = []
batch_size = 100
total_records = len(data)
start_index = 0

while start_index < total_records:
    end_index = start_index + batch_size
    batch_data = data['combined_text'][start_index:end_index]
    results = sentiment_pipeline([str(i) for i in batch_data])

    for result in results:
        score = result['score']
        if score >= threshold_strongly_pro_palestine:
            predicted_labels.append("strongly pro-Palestine")
        elif score >= threshold_pro_palestine:
            predicted_labels.append("pro-Palestine")
        elif score >= threshold_neutral:
            predicted_labels.append("neutral")
        elif score >= threshold_pro_israel:
            predicted_labels.append("pro-Israel")
        else:
            predicted_labels.append("strongly pro-Israel")

    start_index = end_index

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [ ]:
data['predicted_label'] = predicted_labels
for txt, pred_label, rslt in zip(data['combined_text'], predicted_labels, results):
    print(f'Text: "{txt}"\nPredicted Label: {pred_label}\nScore: {rslt["score"]}\n{"-" * 40}')

Text: "wild west bedouin hold illegal camel race firing zone"
Predicted Label: neutral
Score: 0.996301531791687
----------------------------------------
Text: "guy want peace thread share opinion either side please would region think good compromise nation"
Predicted Label: strongly pro-Palestine
Score: 0.8690279722213745
----------------------------------------
Text: "feelghoodmusic story update bib"
Predicted Label: strongly pro-Palestine
Score: 0.45949581265449524
----------------------------------------
Text: "old testament new would view bible differently separation god commanded separated man actually book referring voted read instead receive entire word ask many time hear people refer want nothing eager sing christian song blessing connected israel say born engrafted without separate justify taking thing ignoring rest applies thought disclaimer sacrifice ceremony done course know relevant temple standing one reason also jess ultimate"
Predicted Label: strongly pro-Palestine
Scor

as per license for the above section of code 
MIT License

Copyright (c) 2024 Rafał Prus

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

#### final changes, cleaning things that were missed when collecting data

UFC fighter Israel Adesanya was included in the search since his name is Israel, I have removed posts regarding him since there are quite a bit of them and they are not relevant for this project

In [ ]:
mask = ~((data['title'].str.contains('adesanya', case=False, na=False) | data['title'].str.contains('adesanya', case=False, na=False)) |
         (data['selftext'].str.contains('adesanya', case=False, na=False) | data['selftext'].str.contains('adesanya', case=False, na=False)))

filtered_df = data[mask]